In [1]:
%cd ..

d:\Blockchain\data\orai\orchai_validator


In [2]:
from omegaconf import OmegaConf
from labeling.tools import psql_connect
import numpy as np

In [3]:
config = OmegaConf.load("config/etl.yaml")
cur = psql_connect(**config.dest).cursor()

In [4]:
cur.execute(f"select distinct block_height from {config.dest.table} order by block_height;")
bh = cur.fetchall()
bh = np.array([b[0] for b in bh])

In [5]:
bh.shape

(4655,)

In [13]:
diff = np.diff(bh)
diff.sort()

from collections import Counter

c = Counter(diff)
c

Counter({34: 19,
         60: 1,
         116: 18,
         150: 4608,
         266: 1,
         300: 2,
         390: 1,
         195750: 1,
         195900: 1,
         230550: 1,
         530400: 1})

## Missing block:
```
7272731 7272972 | 241
7883969 7884210 | 241
8699979 8700001 | 22
9057005 9057016 | 11
9057065 9057076 | 11
```

In [7]:
diff = np.diff(bh)
combine_win_size = config.hp.etl.combine_win_size

for i, d in enumerate(diff):
    if d != combine_win_size:
        print(bh[i], bh[i+1], "|", d)

7202771 7203071 | 300
7234871 7234905 | 34
7234905 7235021 | 116
7235021 7235055 | 34
7235055 7235171 | 116
7235171 7235205 | 34
7235205 7235321 | 116
7235321 7235355 | 34
7235355 7235471 | 116
7235471 7235505 | 34
7235505 7235621 | 116
7235621 7235655 | 34
7235655 7235771 | 116
7235771 7235805 | 34
7235805 7235921 | 116
7235921 7235955 | 34
7235955 7236071 | 116
7236071 7236105 | 34
7236105 7236221 | 116
7236221 7236255 | 34
7236255 7236371 | 116
7236371 7236405 | 34
7236405 7236521 | 116
7236521 7236555 | 34
7236555 7236671 | 116
7236671 7236705 | 34
7236705 7236821 | 116
7236821 7236855 | 34
7236855 7236971 | 116
7236971 7237005 | 34
7237005 7237121 | 116
7237121 7237155 | 34
7237155 7237271 | 116
7237271 7237305 | 34
7237305 7237421 | 116
7237421 7237455 | 34
7237455 7237571 | 116
7237571 7237605 | 34
7237605 7237871 | 266
7272371 7272671 | 300
7272671 7273061 | 390
7307411 7307471 | 60
7411721 7942121 | 530400
8046371 8242271 | 195900
8311721 8542271 | 230550
8646521 8842271 | 195

## Total block

In [8]:
cur.execute(f"select min(block_height) from public.validator_block;")
print(cur.fetchall())

[(7049472,)]


In [9]:
config.hp.etl

{'accept_rate': 0.1, 'concentration_level': 0.9, 'vote_score': 2, 'propose_score': 6, 'A': 9, 'B': 4, 'C': 2, 'D': 4, 'vote_proposed_win_size': 14400, 'combine_win_size': 150, 'label_win_size': 100800}

In [10]:
print("total blocks:", ((9070703 - 100800 - 150) - (7049472 + 14400 - 1)) / 150 + 1)

total blocks: 12706.88


## Next block

In [11]:
def next_start(height, step=1):
    return height + config.hp.etl.combine_win_size * step

In [48]:
next_start(8911721, 1) 

8911871

In [38]:
bh[-1]

8911721